In [ ]:
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import LSTM, AveragePooling1D, Conv1D, Reshape, MaxPooling1D, GlobalAveragePooling1D, ZeroPadding1D
from tensorflow.keras.layers import Dense, Flatten,concatenate

from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

from keras.models import load_model
from keras.models import Sequential, Model

In [ ]:
image_size = 224
nClass=11
w=7
h=2048

In [ ]:
def Model_3(w,h,nClass):
    input_layer1 = Input(shape=(w,h))
    
    input_layer2 = Input(shape=(w,h))
    conv2 = ZeroPadding1D(padding=1)(input_layer2)
    conv2 = Conv1D(filters=1024,kernel_size=3,strides=1)(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Dropout(0.2)(conv2)
    
    add = concatenate([input_layer2,input_layer1])
    dense = BatchNormalization()(add)
    dense = GlobalAveragePooling1D()(dense)
    dense = Dense(1024,activation='relu')(dense)
    dense = Dropout(0.2)(dense)
    dense = BatchNormalization()(dense)
    output_layer = Dense(nClass, activation='softmax')(dense)
    model = Model(inputs=[input_layer1,input_layer2], outputs=output_layer)

    return model

In [ ]:
def fitModel2Input(X1_train,X2_train, X1_test, X2_test, y_train, y_test,epochs,model,fileModel):
    learning_rate = 0.01
    decays = learning_rate/epochs
    #decays = learning_rate*math.pow(drop,math.floor((1+epochs)/epochs_drop))
    #decays=1e-6
   # decays = 0.001
    opt = SGD(lr=learning_rate, decay=decays, momentum=0.9, nesterov=False)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

    reduce_lr = ReduceLROnPlateau(monitor='val_loss',min_lr=0.001,factor=0.1,patience=5)
    checkpointer = ModelCheckpoint(filepath=fileModel, verbose=1, save_best_only=True)

    history=model.fit([X1_train,X2_train], y_train, batch_size=32, epochs=epochs,
                                  validation_data=([X1_test,X2_test], y_test), verbose=1,
                                  callbacks=[reduce_lr,checkpointer])
    print("Evaluate model on test data")
    results = model.evaluate([X1_test,X2_test], y_test, batch_size=32)
    print("test loss, test acc:", results)

In [ ]:
def reportAcc(y_train,y_test,X1_test,X2_test,model):
    y_trainc = np.argmax(y_train,axis=1)
    y_testc = np.argmax(y_test,axis=1)
    y_predc = np.argmax(model.predict([X1_test,X2_test]),axis=1)
    print(accuracy_score(y_testc, y_predc)*100)

In [ ]:
def loadData_(dataFile):
    
    x = np.load(dataFile)
    
    return x

def loadLabel_():
    
    labelFile = "feature_food100cut/label_food100.npy"
    y = np.load(labelFile)

    
    return y

In [ ]:
def split_data(x,y,size):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=size)
    return X_train, X_test, y_train, y_test

In [ ]:
featureConv1D = "feature_food100cut/feature_food100_conv1d.npy"
featureLSTM = "feature_food100cut/feature_food100_lstm.npy"

xDataFatureConv1D = loadData_(featureConv1D)
xDataFatureLSTM = loadData_(featureLSTM)
yLabel = loadLabel_()
xConv1D_train, xConv1D_test, y_train, y_test = split_data(xDataFatureConv1D,yLabel,0.1)
xLSTM_train, xLSTM_test, y_train, y_test = split_data(xDataFatureLSTM,yLabel,0.1)

In [ ]:
fileModel = 'model/food100_astffB3.hdf5'
model=Model_3(w,h,nClass)
fitModel2Input(xConv1D_train,xLSTM_train, xConv1D_test, xLSTM_test, y_train, y_test,epochs,model,fileModel)

In [ ]:
fileModel = 'model/food100_astffB3.hdf5'
model = load_model(fileModel)
reportAcc(y_train,y_test,xConv1D_test,xLSTM_test,model)